# Develop Model
In this noteook, we will go through the steps to load the ResNet18 model, pre-process the images to the required format and call the model to find the top predictions.

    Note: Always make sure you don't have any lingering notebooks running (Shutdown previous notebooks). Otherwise it may cause GPU memory issue.

In [ ]:
import numpy as np
from PIL import Image
import os
import wget
import torch
from azureml.core.workspace import Workspace
from dotenv import set_key, find_dotenv
from testing_utilities import get_auth
from azureml.core.model import Model
from fastai.vision import open_image 
import torchvision.models as models
from utils_cv.common.deployment import generate_yaml
from utils_cv.common.data import data_path, root_path 
from utils_cv.common.image import ims2strlist
from utils_cv.classification.model import IMAGENET_IM_SIZE, model_to_learner

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

## Create the model

In [ ]:
learn = model_to_learner(models.resnet18(pretrained=True), IMAGENET_IM_SIZE)
print("model loaded")

In [ ]:
wget.download(
    "https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg"
)

In [ ]:
img_path = "220px-Lynx_lynx_poing.jpg"
pil_img  = Image.open(img_path)
print(pil_img.size)
pil_img

Below, we load the image using the *open_image* function from fastai.

In [ ]:
# Evaluate the model using the input data
img = open_image(img_path)

Now, let's call the model on our image. This will take a few seconds.

In [ ]:
%%time
pred_class, pred_idx, outputs = learn.predict(img)
decoded_predictions = {"label": str(pred_class), "probability": str(outputs[pred_idx].item())}
print("Predicted:", decoded_predictions)

## Register the model
Register an existing trained model, add descirption and tags.

In [ ]:
# Get workspace
# Load existing workspace from the config file info.

ws = Workspace.from_config(auth=get_auth())
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep="\n")

In [ ]:
current_directory = os.getcwd()
output_folder = os.path.join(current_directory, 'outputs')
MODEL_NAME = 'im_classif_resnet18'  # Name we will give our model both locally and on Azure
PICKLED_MODEL_NAME = MODEL_NAME + '.pkl'
os.makedirs(output_folder, exist_ok=True)
learn.export(os.path.join(output_folder, PICKLED_MODEL_NAME))

Register the model

In [ ]:
model = Model.register(
    model_path=os.path.join(output_folder, PICKLED_MODEL_NAME),  # this points to a local file
    model_name="resnet_model",  # this is the name the model is registered as
    tags={"model": "dl", "framework": "resnet"},
    description="resnet model",
    workspace=ws,
)

In [ ]:
print(model.name, model.description, model.version)

In [ ]:
set_key(env_path, "model_version", str(model.version))

In [ ]:
# Clear GPU memory
learn.purge()
torch.cuda.empty_cache()

We have registred the trained ResNet18 model in Azure ML. We can now move on to [developing the model api for our model](02_DevelopModelDriver.ipynb).